In [38]:
#Clusterização da amostra total

In [39]:
#  Importações e Configuração Inicial
import numpy as np
import pandas as pd
import sklearn
import tensorflow as tf
from sklearn.cluster import MiniBatchKMeans
import matplotlib.pyplot as plt
import folium
import matplotlib.colors as mcolors

# Verificar dispositivos TensorFlow
print(f"TensorFlow has access to the following devices:\n{tf.config.list_physical_devices()}")
print(f"TensorFlow version: {tf.__version__}")


TensorFlow has access to the following devices:
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]
TensorFlow version: 2.16.1


In [40]:
 #Carregar dados
df_total = pd.read_csv('../data/amostra_total.csv', sep=';')
print("Dados carregados:")
print(df_total.head())


Dados carregados:
        INDICE   LATITUDE  LONGITUDE CODIGO_ROTA  SEQUENCIA  \
0  401103972.0 -22.858956 -43.338632      70_324     3360.0   
1  402281829.0 -22.830781 -43.395010       88_48      309.0   
2  400507411.0 -22.821902 -43.415018       74_46      254.0   
3  400533412.0 -22.834372 -43.410329       75_57       80.0   
4  400515070.0 -22.861150 -43.338667      70_324     3180.0   

             LOGRADOURO NUMERO  
0  MONS INACIO DA SILVA    545  
1           MARIA ELIZA     80  
2           DONA LURDES      4  
3      LUCIO JOSE FILHO     81  
4         MAR DEL PLATA    490  


In [41]:
# Clusterização inicial com MiniBatch K-means
coordinates = df_total[['LATITUDE', 'LONGITUDE']]
kmeans = MiniBatchKMeans(n_clusters=300, random_state=42, batch_size=10000, n_init=10)
kmeans.fit(coordinates)
df_total['Cluster'] = kmeans.labels_

# Verificar a distribuição dos clusters
cluster_distribution = df_total['Cluster'].value_counts()
print("Distribuição dos clusters:")
print(cluster_distribution.head(10))
print(df_total['Cluster'].value_counts())


Distribuição dos clusters:
Cluster
71     2404
88     2355
15     2310
61     2303
36     2227
267    2172
214    2166
12     2155
226    2092
6      1944
Name: count, dtype: int64
Cluster
71     2404
88     2355
15     2310
61     2303
36     2227
       ... 
211     536
296     530
186     529
52      489
212     334
Name: count, Length: 300, dtype: int64


In [42]:
print(df_total['Cluster'].value_counts())


Cluster
71     2404
88     2355
15     2310
61     2303
36     2227
       ... 
211     536
296     530
186     529
52      489
212     334
Name: count, Length: 300, dtype: int64


In [43]:

mapa = folium.Map(location=[df_total['LATITUDE'].mean(), df_total['LONGITUDE'].mean()], zoom_start=10)
colors = plt.get_cmap('viridis')(np.linspace(0, 1, 300))
colors = [mcolors.rgb2hex(color) for color in colors]

for _, row in df_total.iterrows():
    folium.CircleMarker(
        location=[row['LATITUDE'], row['LONGITUDE']],
        radius=5,
        color=colors[int(row['Cluster'])],
        fill=True
    ).add_to(mapa)




In [44]:
# Adicionar rotas e dias aos clusters e exibir o DataFrame atualizado
def add_routes_and_days(df, clusters_column='Cluster', num_days=22):
    df['Route'] = -1
    df['Day'] = -1
    for cluster_id in df[clusters_column].unique():
        cluster_data = df[df[clusters_column] == cluster_id].copy()
        daily_kmeans = MiniBatchKMeans(n_clusters=num_days, random_state=42)
        cluster_data['Day'] = daily_kmeans.fit_predict(cluster_data[['LATITUDE', 'LONGITUDE']])
        df.loc[cluster_data.index, 'Route'] = cluster_data.index
        df.loc[cluster_data.index, 'Day'] = cluster_data['Day']
    return df

df_total = add_routes_and_days(df_total)

# Ordenar o DataFrame pelas colunas 'Route' e 'Day'
df_total_sorted = df_total.sort_values(by=['Route', 'Day'])

# Exibir o DataFrame atualizado
print("DataFrame com rotas e dias adicionados e ordenados:")
print(df_total_sorted.head())


DataFrame com rotas e dias adicionados e ordenados:
        INDICE   LATITUDE  LONGITUDE CODIGO_ROTA  SEQUENCIA  \
0  401103972.0 -22.858956 -43.338632      70_324     3360.0   
1  402281829.0 -22.830781 -43.395010       88_48      309.0   
2  400507411.0 -22.821902 -43.415018       74_46      254.0   
3  400533412.0 -22.834372 -43.410329       75_57       80.0   
4  400515070.0 -22.861150 -43.338667      70_324     3180.0   

             LOGRADOURO NUMERO  Cluster  Route  Day  
0  MONS INACIO DA SILVA    545      164      0    9  
1           MARIA ELIZA     80      287      1   21  
2           DONA LURDES      4       50      2    9  
3      LUCIO JOSE FILHO     81      208      3   16  
4         MAR DEL PLATA    490       57      4    4  


In [45]:
# Visualização dos subclusters em um mapa (otimizada com logs)
import time

start_time = time.time()
sub_map = folium.Map(location=[df_total['LATITUDE'].mean(), df_total['LONGITUDE'].mean()], zoom_start=12)
sub_colors = plt.get_cmap('plasma')(np.linspace(0, 1, 22))
sub_colors = [mcolors.rgb2hex(color) for color in sub_colors]

print("Iniciando a adição de marcadores ao mapa...")

for i, (_, row) in enumerate(df_total.iterrows()):
    if i % 1000 == 0:
        print(f"Processando ponto {i}/{len(df_total)}...")
    folium.CircleMarker(
        location=[row['LATITUDE'], row['LONGITUDE']],
        radius=5,
        color=sub_colors[int(row['Day'])],
        fill=True,
        popup=f'Subcluster: {row["Day"]}'
    ).add_to(sub_map)

print("Todos os marcadores foram adicionados. Salvando o mapa...")

end_time = time.time()



Iniciando a adição de marcadores ao mapa...
Processando ponto 0/378299...
Processando ponto 1000/378299...
Processando ponto 2000/378299...
Processando ponto 3000/378299...
Processando ponto 4000/378299...
Processando ponto 5000/378299...
Processando ponto 6000/378299...
Processando ponto 7000/378299...
Processando ponto 8000/378299...
Processando ponto 9000/378299...
Processando ponto 10000/378299...
Processando ponto 11000/378299...
Processando ponto 12000/378299...
Processando ponto 13000/378299...
Processando ponto 14000/378299...
Processando ponto 15000/378299...
Processando ponto 16000/378299...
Processando ponto 17000/378299...
Processando ponto 18000/378299...
Processando ponto 19000/378299...
Processando ponto 20000/378299...
Processando ponto 21000/378299...
Processando ponto 22000/378299...
Processando ponto 23000/378299...
Processando ponto 24000/378299...
Processando ponto 25000/378299...
Processando ponto 26000/378299...
Processando ponto 27000/378299...
Processando ponto

In [ ]:
# Salvar DataFrame final
df_total.to_csv('../results/clustered_routes_days.csv', index=False)
print("DataFrame final salvo como 'clustered_routes_days.csv'")



DataFrame final salvo como 'clustered_routes_days.csv'
